<div style="background-color:#b5add6; padding: 1em; color: black;">
    NGROK – Déployer vos applications sur un server par connexion SSH et github action
</div>

Cet notebook a été rédigé grace au sources suivantes: 
* <a href="https://github.com/appleboy/ssh-action">https://github.com/appleboy/ssh-action</a>

In [ ]:
name: Agent Widget CI/CD

on:
  push:
    branches:
      - main
  workflow_dispatch:

jobs:
  CI:
    runs-on: ubuntu-latest
    
    steps:
      - name: Checkout repository
        uses: actions/checkout@v2

      - name: Set up Node.js
        uses: actions/setup-node@v3
        with:
          node-version: '20.19.1'

      - name: Install dependencies
        run: |
          cd agentchat
          npm install

      - name: Build Vue app
        run: |
          cd agentchat
      #    npm run build

      # Remove testing due to hybrid aspect of server (vue + node)
      # - name: Run tests
      #   run: |
      #     cd agent-widget
      #     npm run check-lint

  CD:
    runs-on: ubuntu-latest
    needs: CI
    if: github.ref == 'refs/heads/main' && contains(github.event.head_commit.message, 'deploy')

    steps:
      - name: Checkout repository
        uses: actions/checkout@v2

      - name: Deploy to Server with SSH
        uses: appleboy/ssh-action@master
        # env:
        #   HOST_PORT: ${{ secrets.HOST_PORT }}
        with:
          timeout: 90m
          host: ${{ secrets.DEPLOY_HOST }}
          port: ${{ secrets.DEPLOY_PORT }}
          username: ${{ secrets.DEPLOY_USER }}
          password: ${{ secrets.DEPLOY_PASSWORD }}
          # envs: HOST_PORT
          script: |
            cd ~/serenity-ia
            git pull
            docker-compose down

            echo "[+] Building docker-compose services..."
            # docker-compose build | tee build.log &
            docker-compose build --progress=plain | tee build.log &

            # Wait loop to print output every few seconds to avoid silent timeout
            BUILD_PID=$!
            while kill -0 $BUILD_PID 2> /dev/null; do
              echo "[+] Still building at $(date)..."
              sleep 60
            done

            echo "[+] Build finished. Starting services..."
            docker-compose up -d
          # script: |
          #   cd ~/serenity-ia
          #   git pull
          #   docker-compose down
          #   docker-compose build
          #   docker-compose up -d

# docker-compose build --no-cache

flemme... c'est si clair/ It's so obvious...